In [318]:
import pandas as pd
import sqlalchemy
import math
import os, sys, copy, random, pickle, cv2
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

sys.path.append("../")

%load_ext autoreload
%autoreload 2

from modules.ControlTracking import TrackFactory
from modules.ControlDisplay import Display
from modules.Interproc import DBInterface, GuiviewState

from modules.AnalysisHelpers import ( 
         exploreImgs, buildImgComparisonData, subprocEval)

from modules.EvalHelpers import EvalTracker, EvalDataset
from modules.ControlEval import EvalFactory
from modules.DataSchemas import ScoreSchema
from modules.EvalHelpers import OutcomeData, DFHelper

from modules.EvalHelpers import AggEval, DFHelper

from modules.EvalSuite import EvalSuite

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Build OutcomeData from eval to work with

In [319]:
#different trackers here
b_log, b_eval_log = False, False
vidfn = 'data/proc/tmp/dec14/output5.proc1.proc1.avi'
outcome0 = subprocEval(f_pathfn = vidfn, algo_enum=0, b_log=b_log, b_eval_log=b_eval_log)
outcome1 = subprocEval(f_pathfn = vidfn, algo_enum=1, b_log=b_log, b_eval_log=b_eval_log)
outcome2 = subprocEval(f_pathfn = vidfn, algo_enum=2, b_log=b_log, b_eval_log=b_eval_log)

### Show some basic reports from CmpAlgoReport

In [320]:
from modules.EvalSuite import CmpAlgoReport

In [321]:
cmpA = CmpAlgoReport(current_outcome_data=outcome2, 
                     benchmark_outcome_data=outcome0
                    )

In [322]:
cmpA.basic3()

,improvements,deprovements,sames
calc\nBaseline\nBall\nUnits\nAway\n,3,7,9
check\nBaseline\nInside\nTrack\n,5,0,14
check\nBoth\nContains\nOther\n,3,0,16
check\nEither\nContains\nOther\n,4,0,15
check\nTrack\nInside\nBaseline\n,2,0,17
check\nTrack\nInside\nBaseline\nRect\n,0,0,19
check\nTrack\nSuccess\n,0,0,19
compare\nRadii\n,9,0,10
distance\nFrom\nBaseline\n,2,6,11
prop\nBaseline\nRadius\n,0,0,19


In [324]:
cmpA.basic2()

diff                                   \
    calc\nBaseline\nBall\nUnits\nAway\n check\nBaseline\nInside\nTrack\n   
0                                  0.00                              0.0   
9                                  0.00                              0.0   
78                                 0.05                              0.0   
116                                0.00                              0.0   
136                                0.00                              0.0   
176                               -0.03                              0.0   
187                               -0.14                              1.0   
194                                0.01                              0.0   
195                               -0.12                              1.0   
201                                0.00                              0.0   
215                               -0.29                              1.0   
246                                0.00                              0.0   
270                                0.00                              0.0   
292                                0.00                              0.0   
295                               -0.44                              1.0   
307                               -0.05                              0.0   
308                                0.02                              0.0   
324                               -0.02                              1.0   
340                                0.00                              0.0   

                                                                     \
    check\nBoth\nContains\nOther\n check\nEither\nContains\nOther\n   
0                              0.0                              0.0   
9                              0.0                              0.0   
78                             0.0                              0.0   
116                            0.0                              0.0   
136                            0.0                              0.0   
176                            0.0                              0.0   
187                            0.0                              1.0   
194                            0.0                              0.0   
195                            1.0                              1.0   
201                            0.0                              0.0   
215                            1.0                              1.0   
246                            0.0                              0.0   
270                            0.0                              0.0   
292                            0.0                              0.0   
295                            0.0                              1.0   
307                            0.0                              0.0   
308                            0.0                              0.0   
324                            1.0                              0.0   
340                            0.0                              0.0   

                                                                             \
    check\nTrack\nInside\nBaseline\n check\nTrack\nInside\nBaseline\nRect\n   
0                                0.0                                    0.0   
9                                0.0                                    0.0   
78                               0.0                                    0.0   
116                              0.0                                    0.0   
136                              0.0                                    0.0   
176                              0.0                                    0.0   
187                              0.0                                    0.0   
194                              0.0                                    0.0   
195                              1.0                                    0.0   
201                              0.0                                    0.0  

In [317]:
cmpA.basic()

diff                                 \
                                    max    mean  median    min     std   
agg_calcBaselineBallUnitsAway     0.027  -0.053  -0.053  0.015  -0.035   
agg_checkBaselineInsideTrack        nan   0.263     nan    nan     nan   
agg_checkBothContainsOther          nan   0.158     nan    nan     nan   
agg_checkEitherContainsOther        nan   0.211     nan    nan     nan   
agg_checkTrackInsideBaseline        nan   0.105     nan    nan     nan   
agg_checkTrackInsideBaselineRect    nan   0.000     nan    nan     nan   
agg_checkTrackSuccess               nan   0.000     nan    nan     nan   
agg_compareRadii                  0.000  -0.316   0.000  0.000   0.088   
agg_distanceFromBaseline          0.323  -0.366  -1.143  0.000  -0.214   
agg_propBaselineRadius            0.000   0.000   0.000  0.000   0.000   
agg_propTrackRadius               0.000   0.551   1.000  0.000   0.052   

                                          current                        \
                                      sum     max    mean median    min   
agg_calcBaselineBallUnitsAway      -1.003   1.044   0.407  0.395  0.043   
agg_checkBaselineInsideTrack          nan     nan   0.684    nan    nan   
agg_checkBothContainsOther            nan     nan   0.579    nan    nan   
agg_checkEitherContainsOther          nan     nan   0.737    nan    nan   
agg_checkTrackInsideBaseline          nan     nan   0.632    nan    nan   
agg_checkTrackInsideBaselineRect      nan     nan   0.105    nan    nan   
agg_checkTrackSuccess                 nan     nan   1.000    nan    nan   
agg_compareRadii                   -6.000   7.000   2.000  2.000  0.000   
agg_distanceFromBaseline           -6.948  12.530   5.125  4.243  1.000   
agg_propBaselineRadius              0.000  26.000   9.842  6.000  3.000   
agg_propTrackRadius               190.000  33.000  12.928  9.000  1.000   

                                                    benchmark                 \
                                     std        sum       max    mean median   
agg_calcBaselineBallUnitsAway      0.293      7.738     1.017   0.460  0.449   
agg_checkBaselineInsideTrack         nan        nan       nan   0.421    nan   
agg_checkBothContainsOther           nan        nan       nan   0.421    nan   
agg_checkEitherContainsOther         nan        nan       nan   0.526    nan   
agg_checkTrackInsideBaseline         nan        nan       nan   0.526    nan   
agg_checkTrackInsideBaselineRect     nan        nan       nan   0.105    nan   
agg_checkTrackSuccess                nan        nan       nan   1.000    nan   
agg_compareRadii                   1.915     38.000     7.000   2.316  2.000   
agg_distanceFromBaseline           2.914     97.384    12.207   5.491  5.385   
agg_propBaselineRadius             8.187    187.000    26.000   9.842  6.000   
agg_propTrackRadius               10.414  4,460.000    33.000  12.377  8.000   

                                                            
                                    min     std        sum  
agg_calcBaselineBallUnitsAway     0.028   0.328      8.742  
agg_checkBaselineInsideTrack        nan     nan        nan  
agg_checkBothContainsOther          nan     nan        nan  
agg_checkEitherContainsOther        nan     nan        nan  
agg_checkTrackInsideBaseline        nan     nan        nan  
agg_checkTrackInsideBaselineRect    nan     nan        nan  
agg_checkTrackSuccess               nan     nan        nan  
agg_compareRadii                  0.000   1.827     44.000  
agg_distanceFromBaseline          1.000   3.128    104.332  
agg_propBaselineRadius            3.000   8.187    187.000  
agg_propTrackRadius               1.000  10.362  4,270.000

##### Build a shared table

In [126]:
benchmark_df = cmpA.benchmark.aggDfh.df.copy()
current_df   = cmpA.current.aggDfh.df.copy()

In [129]:
diff_values = current_df.values - benchmark_df.values
diff_df = pd.DataFrame(diff_values, columns=current_df.columns, index=current_df.index)
diff_df

,max,mean,median,min,std,sum
agg_calcBaselineBallUnitsAway,0.026951,-0.052811,-0.053479,0.014717,-0.034736,-1.003418
agg_checkBaselineInsideTrack,NaN,0.263158,NaN,NaN,NaN,NaN
agg_checkBothContainsOther,NaN,0.157895,NaN,NaN,NaN,NaN
agg_checkEitherContainsOther,NaN,0.210526,NaN,NaN,NaN,NaN
agg_checkTrackInsideBaseline,NaN,0.105263,NaN,NaN,NaN,NaN
agg_checkTrackInsideBaselineRect,NaN,0.000000,NaN,NaN,NaN,NaN
agg_checkTrackSuccess,NaN,0.000000,NaN,NaN,NaN,NaN
agg_compareRadii,0.000000,-0.315789,0.000000,0.000000,0.087512,-6.000000
agg_distanceFromBaseline,0.323408,-0.365708,-1.142524,0.000000,-0.213608,-6.948443
agg_propBaselineRadius,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [28]:
#different trackers here
vidfn = 'data/proc/tmp/dec14/output5.proc1.proc1.avi'
outcome0 = subprocEval(f_pathfn = vidfn, algo_enum=0, b_log=True, b_eval_log=True)
outcome1 = subprocEval(f_pathfn = vidfn, algo_enum=1, b_log=True, b_eval_log=True)
outcome2 = subprocEval(f_pathfn = vidfn, algo_enum=2, b_log=True, b_eval_log=True)


START - running eval module...
[                            ]
-----------------------------
output db: data/usr/eval_tmp.db
output tbl: outcome_dataframe
eval time: 4.493
outcome rows: 345 cols: 48 

num frames:                  345 
obj enums scored/tracked:    0,1 / 0 
num scored frames:           19 | 0,9,78...308,324,340 
-------
agg_calcBaselineBallUnitsAway   0.46
agg_checkBothContainsOther      0.42
agg_checkTrackSuccess           1.00
fagg_less_than_20_pix_balls_away   0.53
fagg_less_than_30_pix_success      1.00
-----
subproc time: 6.176

START - running eval module...
[                            ]
-----------------------------
output db: data/usr/eval_tmp.db
output tbl: outcome_dataframe
eval time: 4.521
outcome rows: 345 cols: 48 

num frames:                  345 
obj enums scored/tracked:    0,1 / 0 
num scored frames:           19 | 0,9,78...308,324,340 
-------
agg_calcBaselineBallUnitsAway   0.46
agg_checkBothContainsOther      0.42
agg_checkTrackSuccess           1.0

### Pandas ScratchPad

##### Explore the mini-df

In [73]:
mini = cmpA.benchmark.aggFilterDfh.df
mini

,mean,n
fagg_less_than_20_pix_balls_away,0.533238,16.0
fagg_less_than_30_pix_success,1.000000,310.0


In [74]:
mini_stack = mini.stack()
mini_stack

fagg_less_than_20_pix_balls_away  mean      0.533238
                                  n        16.000000
fagg_less_than_30_pix_success     mean      1.000000
                                  n       310.000000
dtype: float64

In [75]:
mini.index

Index([u'fagg_less_than_20_pix_balls_away', u'fagg_less_than_30_pix_success'], dtype='object')

In [76]:
mini_stack.index

MultiIndex(levels=[[u'fagg_less_than_20_pix_balls_away', u'fagg_less_than_30_pix_success'], [u'mean', u'n']],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [77]:
mini_stack.T

fagg_less_than_20_pix_balls_away  mean      0.533238
                                  n        16.000000
fagg_less_than_30_pix_success     mean      1.000000
                                  n       310.000000
dtype: float64

##### Easiest Way to combine two dataframes and note them as distinct

In [79]:
#on axis=1, dif-df's are ordered by col, so columns are MutliIndex
axis = 1

d_tmp = {'current':mini, 'benchmark':mini}

df_concat = pd.concat(d_tmp.values(), keys=d_tmp.keys(), axis=axis)

print 'cols:  %s', str(df_concat.columns)
print 'index: %s', str(df_concat.index)

df_concat

cols:  %s MultiIndex(levels=[[u'current', u'benchmark'], [u'mean', u'n']],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])
index: %s Index([u'fagg_less_than_20_pix_balls_away', u'fagg_less_than_30_pix_success'], dtype='object')


current        benchmark       
                                      mean      n      mean      n
fagg_less_than_20_pix_balls_away  0.533238   16.0  0.533238   16.0
fagg_less_than_30_pix_success     1.000000  310.0  1.000000  310.0

In [80]:
# on axis=0, df's are ordered by row, so rows are MultiIndex
axis = 0

d_tmp = {'current':mini, 'benchmark':mini}

df_concat = pd.concat(d_tmp.values(), keys=d_tmp.keys(), axis=axis)

print 'cols:  %s', str(df_concat.columns)
print 'index: %s', str(df_concat.index)

df_concat

cols:  %s Index([u'mean', u'n'], dtype='object')
index: %s MultiIndex(levels=[[u'current', u'benchmark'], [u'fagg_less_than_20_pix_balls_away', u'fagg_less_than_30_pix_success']],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])


mean      n
current   fagg_less_than_20_pix_balls_away  0.533238   16.0
          fagg_less_than_30_pix_success     1.000000  310.0
benchmark fagg_less_than_20_pix_balls_away  0.533238   16.0
          fagg_less_than_30_pix_success     1.000000  310.0

##### here's an attempt to combine dataframes with manually created MultiIndex

In [ ]:
iters = [list(mini.index), ['first']]
multi_ind_1 = pd.MultiIndex.from_product(iters, names=('criteria','dfnum'))
mini1 = pd.DataFrame(mini, index=multi_ind_1)
mini1

In [69]:
iters = [list(mini.index), ['second']]
multi_ind_2 = pd.MultiIndex.from_product(iters, names=('criteria','dfnum'))
mini2 = pd.DataFrame(data=mini.values, index=multi_ind_2, columns=mini.columns)

mini2

,,mean,n
criteria,dfnum,,
fagg_less_than_20_pix_balls_away,second,0.533238,16.0
fagg_less_than_30_pix_success,second,1.000000,310.0


In [ ]:
iters = [list(mini.index), ['first','second']]
multi_ind = pd.MultiIndex.from_product(iters, names=('criteria','dfnum'))
multi_ind